In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import time
import os
import math

In [2]:
# Model Hyperparameters
flags=tf.flags

flags.DEFINE_string('word2vec_norm','embeddings/word2vec_norm.txt','Word2vec file with pre-trained embeddings')
flags.DEFINE_string('data_path','SICK','SICK data set path')
flags.DEFINE_string('save_path','SICK/STS_log','STS model output directory')
flags.DEFINE_integer('embedding_dim',300,'Dimensionality of word embedding')
flags.DEFINE_integer('max_length',26,'one sentence max length words which is in dictionary')
flags.DEFINE_bool('use_fp64',False,'Train using 64-bit floats instead of 32bit floats')

FLAGS=flags.FLAGS
FLAGS._parse_flags()
print('Parameters:')
for attr,value in sorted(FLAGS.__flags.items()):
    print('{}={}'.format(attr,value))

Parameters:
data_path=SICK
embedding_dim=300
max_length=26
save_path=SICK/STS_log
use_fp64=False
word2vec_norm=embeddings/word2vec_norm.txt


In [3]:
def data_type():
    return tf.float64 if FLAGS.use_fp64 else tf.float32

In [4]:
def build_vocab(word2vec_path=None):
    if word2vec_path:
        print('Load word2vec_norm file {}'.format(word2vec_path))
        with open(word2vec_path,'r') as f:
            header=f.readline()
            vocab_size,layer2_size=map(int,header.split())
            # initial matrix with random uniform
            init_W=np.random.uniform(-0.25,0.25,(vocab_size,FLAGS.embedding_dim))

            print('vocab_size={}'.format(vocab_size))
            dictionary=dict()
            while True:
                line=f.readline()
                if not line:break
                word=line.split()[0]
                dictionary[word]=len(dictionary)
                init_W[dictionary[word]]=np.array(line.split()[1:], dtype=np.float32)

        return dictionary,init_W

In [5]:
def file_to_word_ids(filename,word_to_id,is_test=False):
    with open(filename,'r') as f:
        f.readline() # remove header
        sentences_A=[]
        sentencesA_length=[]
        sentences_B=[]
        sentencesB_length=[]
        relatedness_scores=[]
        pairIDs=[]
        while True:
            line=f.readline()
            if not line: break
            ID=line.split('\t')[0] # for test
            pairIDs.append(ID)
            sentence_A=line.split('\t')[1]
            sentence_B=line.split('\t')[2]
            relatedness_score=line.split('\t')[3]    
            _=[word_to_id[word] for word in sentence_A.split() if word in word_to_id]
            _+=[0]*(FLAGS.max_length-len(_))
            sentences_A.append(_)
            sentencesA_length.append(len(_))
            _=[word_to_id[word] for word in sentence_B.split() if word in word_to_id]
            _+=[0]*(FLAGS.max_length-len(_))
            sentences_B.append(_)
            sentencesB_length.append(len(_))
            relatedness_scores.append((float(relatedness_score)-1)/4)
    assert len(sentences_A)==len(sentencesA_length)==len(sentences_B)==len(sentencesB_length)==len(relatedness_scores)
    if not is_test: return STSInput(sentences_A,sentencesA_length,sentences_B,sentencesB_length,relatedness_scores)
    else:
        stsinput=STSInput(sentences_A,sentencesA_length,sentences_B,sentencesB_length,relatedness_scores)
        stsinput.pairIDs=pairIDs
        return stsinput

In [6]:
class STSInput(object):
    def __init__(self,sentences_A,sentencesA_length,sentences_B,sentencesB_length,relatedness_scores):
        self.sentences_A=sentences_A
        self.sentencesA_length=sentencesA_length
        self.sentences_B=sentences_B
        self.sentencesB_length=sentencesB_length
        self.relatedness_scores=relatedness_scores
    
    def sentences_A(self):
        return self.sentences_A
    
    def sentencesA_length(self):
        return self.sentencesA_length
    
    def sentences_B(self):
        return self.sentences_B
    
    def sentencesA_length(self):
        return self.sentencesB_length
    
    def relatedness_scores(self):
        return self.relatedness_scores

In [7]:
train_path=os.path.join(FLAGS.data_path,'SICK_train.txt')
valid_path=os.path.join(FLAGS.data_path,'SICK_trial.txt')
test_path=os.path.join(FLAGS.data_path,'SICK_test_annotated.txt')

dictionary,init_W=build_vocab(FLAGS.word2vec_norm)
train_data=file_to_word_ids(train_path,dictionary)
valid_data=file_to_word_ids(valid_path,dictionary)
test_data=file_to_word_ids(test_path,dictionary,is_test=True)

Load word2vec_norm file embeddings/word2vec_norm.txt
vocab_size=2378


In [8]:
def next_batch(start,end,input):
    inputs_A=input.sentences_A[start:end]
    inputsA_length=input.sentencesA_length[start:end]
    inputs_B=input.sentences_B[start:end]
    inputsB_length=input.sentencesB_length[start:end]
    labels=np.reshape(input.relatedness_scores[start:end],(len(range(start,end)),1))
    return STSInput(inputs_A,inputsA_length,inputs_B,inputsB_length,labels)

In [10]:
def build_model(input_,input_length,dropout_):
    rnn_cell=tf.nn.rnn_cell.LSTMCell(num_units=50,forget_bias=0.0,state_is_tuple=True)
    rnn_cell=tf.nn.rnn_cell.DropoutWrapper(rnn_cell,output_keep_prob=dropout_)
    #rnn_cell=tf.nn.rnn_cell.MultiRNNCell([rnn_cell]*50,state_is_tuple=True)
        
    outputs,last_states=tf.nn.dynamic_rnn(
        cell=rnn_cell,
        dtype=data_type(),
        sequence_length=input_length,
        inputs=input_
    )
    return outputs,last_states

In [22]:
class Config(object):
    init_scale=0.1
    learning_rate=10
    max_grad_norm=1
    keep_prob=1.0
    lr_decay=0.9
    batch_size=20
    max_epoch=15
    max_max_epoch=200
    
config=Config()

In [23]:
with tf.Graph().as_default():
    initializer=tf.random_uniform_initializer(-config.init_scale,config.init_scale)
    
    with tf.variable_scope('Model',initializer=initializer):
        sentences_A=tf.placeholder(tf.int32,shape=([None,FLAGS.max_length]),name='sentences_A')
        sentencesA_length=tf.placeholder(tf.int32,shape=([None]),name='sentencesA_length')
        sentences_B=tf.placeholder(tf.int32,shape=([None,FLAGS.max_length]),name='sentences_B')
        sentencesB_length=tf.placeholder(tf.int32,shape=([None]),name='sentencesB_length')
        labels=tf.placeholder(tf.float32,shape=([None,1]),name='relatedness_score_label')
        dropout_f=tf.placeholder(tf.float32)
        W=tf.Variable(tf.constant(0.0,shape=[len(dictionary),FLAGS.embedding_dim]),trainable=False,name='W')
        embedding_placeholder=tf.placeholder(data_type(),[len(dictionary),FLAGS.embedding_dim])
        embedding_init=W.assign(embedding_placeholder)

        sentences_A_emb=tf.nn.embedding_lookup(params=embedding_init,ids=sentences_A)
        sentences_B_emb=tf.nn.embedding_lookup(params=embedding_init,ids=sentences_B)

        with tf.variable_scope('siamese') as scope:
            outputs_A,last_states_A=build_model(sentences_A_emb,sentencesA_length,dropout_f)
            scope.reuse_variables()
            outputs_B,last_states_B=build_model(sentences_B_emb,sentencesB_length,dropout_f)

        last_A=tf.transpose(outputs_A,[1,0,2])[-1]
        last_B=tf.transpose(outputs_B,[1,0,2])[-1]
        #concat_outputs=tf.concat(1,[last_A,last_B])
        #fully_connected = tf.contrib.layers.fully_connected(concat_outputs,num_outputs=1,activation_fn=tf.tanh)
        prediction=tf.exp(tf.mul(-1.0,tf.reduce_mean(tf.abs(tf.sub(last_A,last_B)),1)))

        cost=tf.reduce_mean(tf.square(tf.sub(prediction, labels)))

        lr=tf.Variable(0.0,trainable=False)
        tvars=tf.trainable_variables()
        grads,_=tf.clip_by_global_norm(tf.gradients(cost,tvars),config.max_grad_norm)
        optimizer=tf.train.AdadeltaOptimizer(learning_rate=lr)
        train_op=optimizer.apply_gradients(zip(grads,tvars),global_step=tf.contrib.framework.get_or_create_global_step())
        new_lr=tf.placeholder(tf.float32,shape=[],name='new_learning_rate')
        lr_update=tf.assign(lr,new_lr)
        
        sv=tf.train.Supervisor(logdir=FLAGS.save_path)
        with sv.managed_session() as sess:
            total_batch=int(len(train_data.sentences_A)/config.batch_size)
            print('Total batch size: {}'.format(total_batch))

            # train
            for epoch in range(config.max_max_epoch):
                lr_decay=config.lr_decay**max(epoch+1-config.max_epoch,0.0)
                sess.run([lr,lr_update],feed_dict={new_lr:config.learning_rate*lr_decay})
                print('Epoch {} Learning rate: {}'.format(epoch,sess.run(lr)))
                
                avg_cost=0.
                for i in range(total_batch):
                    start=i*config.batch_size
                    end=(i+1)*config.batch_size

                    next_batch_input=next_batch(start,end,train_data)
                    _,train_cost,train_predict=sess.run([train_op,cost,prediction],feed_dict={
                            sentences_A:next_batch_input.sentences_A,
                            sentencesA_length:next_batch_input.sentencesA_length,
                            sentences_B:next_batch_input.sentences_B,
                            sentencesB_length:next_batch_input.sentencesB_length,
                            labels:next_batch_input.relatedness_scores,
                            dropout_f:config.keep_prob,
                            embedding_placeholder:init_W
                        })
                    avg_cost+=train_cost
                    
                print('Average cost: {}'.format(avg_cost/total_batch))

            # validation
            valid_cost,valid_predict=sess.run([cost,prediction],feed_dict={
                sentences_A:valid_data.sentences_A,
                sentencesA_length:valid_data.sentencesA_length,
                sentences_B:valid_data.sentences_B,
                sentencesB_length:valid_data.sentencesB_length,
                labels:np.reshape(valid_data.relatedness_scores,(len(valid_data.relatedness_scores),1)),
                embedding_placeholder:init_W,
                dropout_f:1.0
            })
            print(valid_cost)
            
            if FLAGS.save_path:
                print('Saving model to {}'.format(FLAGS.save_path))
                sv.saver.save(sess,FLAGS.save_path,global_step=sv.global_step)

            # test
            test_cost,test_predict=sess.run([cost,prediction],feed_dict={
                sentences_A:test_data.sentences_A,
                sentencesA_length:test_data.sentencesA_length,
                sentences_B:test_data.sentences_B,
                sentencesB_length:test_data.sentencesB_length,
                labels:np.reshape(test_data.relatedness_scores,(len(test_data.relatedness_scores),1)),
                embedding_placeholder:init_W,
                dropout_f:1.0
            })
            print(test_cost)

Total batch size: 225
Epoch 0 Learning rate: 10.0
Average cost: 0.120876379849
Epoch 1 Learning rate: 10.0
Average cost: 0.112806319189
Epoch 2 Learning rate: 10.0
Average cost: 0.130850661132
Epoch 3 Learning rate: 10.0
Average cost: 0.140222729726
Epoch 4 Learning rate: 10.0
Average cost: 0.120781012409
Epoch 5 Learning rate: 10.0
Average cost: 0.120872950604
Epoch 6 Learning rate: 10.0
Average cost: 0.12807044213
Epoch 7 Learning rate: 10.0
Average cost: 0.126877213816
Epoch 8 Learning rate: 10.0
Average cost: 0.132872005105
Epoch 9 Learning rate: 10.0
Average cost: 0.139560285509
Epoch 10 Learning rate: 10.0
Average cost: 0.156146437145
Epoch 11 Learning rate: 10.0
Average cost: 0.137624321298
Epoch 12 Learning rate: 10.0
Average cost: 0.148955358565
Epoch 13 Learning rate: 10.0
Average cost: 0.146261130604
Epoch 14 Learning rate: 10.0
Average cost: 0.144788722611
Epoch 15 Learning rate: 9.0
Average cost: 0.146619875067
Epoch 16 Learning rate: 8.10000038147
Average cost: 0.14559383

KeyboardInterrupt: 

In [ ]:
with open('SICK/sts_test_result.txt','w') as fw:
    fw.write('pair_ID	relatedness_score	entailment_judgment\n')
    for _ in range(len(test_predict)):
        fw.write(test_data.pairIDs[_]+'\t'+str(test_predict[_]*4+1)+'\tNA\n')